# 1. Preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import random
import cv2

In [1]:
## Declare Directory
train_dir = "../input/face-mask-12k-images-dataset/Face Mask Dataset/Train"
val_dir = "../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation"
test_dir = "../input/face-mask-12k-images-dataset/Face Mask Dataset/Test"

classes = ["With Mask", "Without Mask"]

In [1]:
n = 5
## Check Image
plt.figure(figsize=(15, n))
for i in range(n):
    # read image
    sample = random.choice(os.listdir(train_dir + "/WithMask"))
    # print("filename:", sample)
    img_dir = train_dir + "/WithMask/" + sample
    img = cv2.imread(img_dir)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # plot image
    plt.subplot(1, n, 1+i)
    plt.imshow(img)
    plt.xlabel("With Mask")
plt.show()   

plt.figure(figsize=(15, n))
for i in range(n):
    # read image
    sample = random.choice(os.listdir(train_dir + "/WithoutMask"))
    # print("filename:", sample)
    img_dir = train_dir + "/WithoutMask/" + sample
    img = cv2.imread(img_dir)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # plot image
    plt.subplot(1, n, 1+i)
    plt.imshow(img)
    plt.xlabel("Without Mask")
plt.show()   

In [1]:
## Data Augmentation 
from keras.preprocessing.image import ImageDataGenerator

In [1]:
# Dataset Loader
train_datagen = ImageDataGenerator(
                                rescale=1./255,
                                rotation_range=0.2,
                                #width_shift_range=0.1,
                                #height_shift_range=0.1,
                                shear_range=0.2,
                                #zoom_range=0.09,
                                horizontal_flip=True,
                                vertical_flip=False,
                                #validation_split=0.1
                                )

val_datagen = ImageDataGenerator(rescale=1./255)

In [1]:
# Image Generator Config
target_size = (150, 150)
batch_size = 16

# Load Dataset
train_dataset = train_datagen.flow_from_directory(train_dir,
                                                  target_size=target_size,
                                                  batch_size=batch_size,
                                                  class_mode="categorical",
                                                  shuffle=True)

val_dataset = val_datagen.flow_from_directory(val_dir,
                                              target_size=target_size,
                                              batch_size=batch_size,
                                              class_mode="categorical",
                                              shuffle=False)

# 2. Deep Learning Model

In [1]:
# Import
import keras
from keras import layers
from keras.applications import MobileNetV2

# Initiate Baseline Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(150, 150, 3))

In [1]:
# Freezing Layer
for layer in base_model.layers:
    layer.trainable = False

In [1]:
model = keras.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(2, activation="softmax"))

In [1]:
model.summary()

In [1]:
## Setting backprop of model (how this model learning)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics ="accuracy")

In [1]:
# Training
EPOCHS = 10
history = model.fit_generator(train_dataset,
                               steps_per_epoch=len(train_dataset)//train_dataset.batch_size,
                               validation_data=val_dataset, 
                               validation_steps=len(val_dataset)//val_dataset.batch_size,
                               epochs=EPOCHS, 
                               )

# 3. Reviewing Model

In [1]:
## Review Our Model
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(14,5))
grid = gridspec.GridSpec(ncols=2,nrows=1,figure=fig)
fig.add_subplot(grid[0])
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

fig.add_subplot(grid[1])
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

#plt.savefig("Training_result.jpg",dpi=300)

In [1]:
# Load Test Dataset
test_dataset = val_datagen.flow_from_directory(test_dir,
                                            target_size=target_size,
                                            batch_size=1,
                                            class_mode=None,
                                            shuffle=False)

In [1]:
probabilities = model.predict_generator(test_dataset)

In [1]:
y_pred = probabilities.argmax(axis=-1)
y_test = test_dataset.classes

In [1]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [1]:
print("Accuracy Score of Model:", accuracy_score(y_pred,y_test))

In [1]:
labels = ["No Mask","Mask"]

fig, ax = plt.subplots(figsize=(8,7))
sns.heatmap(confusion_matrix(y_test,y_pred),xticklabels=labels, ax=ax,
                                       yticklabels=labels, annot=True,fmt="1.0f",cbar=False,annot_kws={"size": 20})
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
plt.title("Confusion matrix",fontsize=30)

# Test with Visualization

In [1]:
import glob
import random

In [1]:
def preprocessing_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (150, 150))
    img = np.array(img)
    img = np.expand_dims(img, axis=0)
    img = img/255
    return img


In [1]:
random_test_img = random.choice(glob.glob(test_dir+"/*/*"))
print(random_test_img)
img_test = cv2.imread(random_test_img)
img_test = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)
plt.imshow(img_test)
plt.show()

In [1]:
img_test = preprocessing_img(img_test)
result = model.predict(img_test)
score = np.max(result)
predicted_class = classes[np.argmax(result)]
print(predicted_class)
print("Confident: ", score)

# Save Model

In [1]:
model.save("face-masked-detection.h5")